This notebook is currently in progress

# GEE Aster Preprocessing

## Step 1: Create a google cloud project

Follow this link to create a google could project: https://cloud.google.com/resource-manager/docs/creating-managing-projects

## Step 2: Installation


In [ ]:
%pip install git+https://github.com/Mining-for-the-Future/ASTER_preprocessing.git

## Step 3: Importing Libraries

In [12]:
import ASTER_preprocessing

In [ ]:
from ASTER_preprocessing import preprocessing

In [7]:
import ee

In [ ]:
%pip install geemap

In [14]:
import geemap

## Step 4: Google Earth Engine Authentication

In [10]:
ee.Authenticate()
ee.Initialize(project="")

## Step 5: Preprocess ASTER Imagery Using a Drawn Polygon

In [ ]:
# Create an interactive map
Map = geemap.Map()

# Add drawing tools to the map
Map.addDrawingTools()

# Display the map
Map

In [ ]:
# Get the last drawn feature
geometry = Map.draw_last_feature

# Check if a geometry was drawn and it's a Polygon
if geometry is not None and geometry.geometry().type().getInfo() == 'Polygon':
    # Run the preprocessing
    result = preprocessing.aster_preprocessing(geometry.geometry())
    # `result` is a dictionary containing the preprocessed image and metadata
else:
    print("Please draw a polygon on the map.")